In [ ]:
import os.path
from bs4 import BeautifulSoup as bs

def get_pairs(file) -> dict:
    # get question
    scrape = bs(file)
    
    questions = scrape.select("div[class='question_text user_content']")
    questions_content = []
    for q in questions:
        questions_content.append(q.text.strip().replace('\n', ''))

    answers = scrape.select("div.correct_answer")
    answers_content = []
    for a in answers:
        answers_content.append(a.text.strip().replace('\n', ''))

    print("Question Count: " + str(len(questions)))
    print(questions_content)
    print("Answer Count: " + str(len(answers)))
    print(answers_content)

    return []


In [ ]:
def write_pairs(pairs: dict, location: str):
    with open(location, 'w', encoding="utf8") as f:
        for key in pairs.keys():
            f.write(f"{key}\t{pairs[key]}\n")

def write_html_file(file, path):
    f = open(path, 'w')
    f.write(file)

def perform_actions(input_path):
    split = os.path.split(input_path)
    output_path = split[0] + "/output/" + split[1].split('.')[0] + "-output.txt"

    file = open(input_path, "r", encoding="utf8").read().strip()
    pairs = get_pairs(file)

    if len(pairs)<1:
        print("Write Failed!")
        return

    write_pairs(pairs, output_path)

    print(f'Wrote to {output_path}.')

def get_HTML_paths_from_directory(dir_path):
    results = []
    for file in os.listdir(dir_path):
        if file.endswith(".html"):
            results.append(os.path.join(dir_path, file))
    return results


In [ ]:
def main():
    # input_path = input("Enter the Input File Path")
    # input_path = "/Users/zak/dev/canvas-to-quizlet/examples/frame1.html"
    # perform_actions(input_path)

    html_paths = get_HTML_paths_from_directory("/Users/zak/dev/canvas-to-quizlet/examples")
    
    for f in html_paths:
        print(f)
        perform_actions(f)
        print("\n")
    

if __name__ == "__main__":
    main()